In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import os
import numpy as np
import codecs
import pickle
import nltk
import pickle
from nltk.classify.maxent import MaxentClassifier, BinaryMaxentFeatureEncoding
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

In [0]:
# Global variables
rawdata_path = "/content/gdrive/My Drive/ml/data/rawdata/"
data_path = "/content/gdrive/My Drive/ml/data/data/"
model_path = "/content/gdrive/My Drive/ml/model/"
labels = ['B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'O']
labels_dict = {labels[i]: i for i in range(len(labels))}
eval_labels = ['B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
print(labels_dict)

{'B-PER': 0, 'I-PER': 1, 'B-ORG': 2, 'I-ORG': 3, 'B-LOC': 4, 'I-LOC': 5, 'O': 6}


In [0]:
def prepare_data(path):
    ''' Create training data and testing data
        Format of data: CoNLL

        Args:
        path: path of data folder
        scale: test size
        index_attri: Represents the number of attributes and the associated attribute type
            index_attri == 1 : The number of attributes = 1 - only ner label. ex: [('Huế', 'B_LOC'), ('là', 'O'), ('thành_phố', 'O'), ('đẹp', 'O')]
            index_attri == 2.1 : The number of attributes = 2(pos-tagging label, ner label). ex: [('Đó', 'P', 'O'), ('là', 'V',  'O'), ('con', 'Nc', 'O'), ('đường', 'N', , 'O')]
            index_attri = 2.2 : The number of attributes = 2(chunking label, ner label). ex: [('Đó', 'B-NP', 'O'), ('là', 'B-VP', 'O'), ('con', 'B-NP', 'O'), ('đường', 'B-NP', 'O')]
            index_attri = 3 : The number of attributes = 3(pos-tagging label,chunking, ner label). ex: [('Đó', 'P', 'B-NP', 'O'), ('là', 'V', 'B-VP', 'O'), ('con', 'Nc', 'B-NP', 'O'), ('đường', 'N', 'B-NP', 'O')]
            if index_attri not in {1,2.1,2,2,3} index_attri = 2.1
        Return:
        train_sents, test_sents
        
        Example of format data:
        [[('Đó', 'P', 'B-NP', 'O'), ('là', 'V', 'B-VP', 'O'), ('con', 'Nc', 'B-NP', 'O'), ('đường', 'N', 'B-NP', 'O')],
        [('Đó', 'P', 'B-NP', 'O'), ('là', 'V', 'B-VP', 'O'), ('con', 'Nc', 'B-NP', 'O'), ('đường', 'N', 'B-NP', 'O')],
    '''    
    list_files = os.listdir(path)
    all_data = []
    ''' Convert data format to CoNll '''
    #training data
    c = 0;
    pos_tag = []
    chunk_tag = []
    ne_tag = []
    for file in list_files:
        with codecs.open(path + file,'r',encoding='utf8') as f:
            sentence = []
            remove = False
            for line in f:
                line = line.split()
                if len(line) > 3:
                    #label_set.append(line[3])
                    if line[3] not in labels:
                        remove = True
                    else:
                        pos_tag.append(line[1])
                        chunk_tag.append(line[2])
                    sentence.append((line[0],line[1],line[2],line[3]))
                else:
                    if len(sentence) > 0:
                        if remove == False:                            
                            all_data.append(sentence)
                        else:
                            remove = False
                        sentence = []
            f.close()

    pos_tag = set(pos_tag)
    chunk_tag = set(chunk_tag)
    return  all_data, pos_tag, chunk_tag

In [0]:
def shape_feature(word):
    is_lower            = 'is_lower'
    is_capital          = 'is_capital' 
    is_title            = 'is_title' 
    is_mix              = 'is_mix' 
    is_capital_period   = 'is_capital_period' 
    is_digit            = 'is_digit' 
    end_digit           = 'end_digit' 
    has_hyphen          = 'has_hyphen' 
    is_code             = 'is_code' 
    num_syllabus        = 'num_syllabus'
    is_name             = 'is_name' 

    check_code = False
    for char in word:
        if char.isdigit():
            check_code = True
            break;

    ft = {
        'bias'                : 1,
        is_lower            : word.islower(),
        is_capital          : word.isupper(),
        is_title            : word.istitle(),
        is_mix              : not(word.islower() and word.isupper()),
        is_capital_period   : (('.' in word) and word[0].isupper()),
        is_digit            : word.isdigit(),
        end_digit           : word[-1].isdigit(),
        has_hyphen          : ('-' in word),
        is_code             : check_code,
        num_syllabus        : (word.count('_') + 1),
        is_name             : word[0].isupper()
    }   
    return ft

def word_feature(sent, i, pre_state, pre_pre_state, sent_re_ft):
    word = sent[i][0]
    ft = dict()
    ### basic feature 
    # current word
    ft['w0'] = word
    # previous entity tag
    ft['s-1'] = pre_state
    ft['s-2'] = pre_pre_state
    ### basic shape feature
    ft.update(shape_feature(word))
    ### basic joint feature
    if i > 0:
        ft['w-1'] = sent[i-1][0]
    else:
        ft['w-1'] = 'BOS'
    
    if i > 1:
        ft['w-2'] = sent[i-2][0]
    else:
        ft['w-2'] = 'BOS'
    if i < len(sent)-1:
        ft['w+1'] = sent[i+1][0]#.replace('_', ' ')
    else:
        ft['w+1'] = 'EOS'
    
    if i < len(sent)-2:
        ft['w+2'] = sent[i+2][0]
    else:
        ft['w+2'] = 'EOS'
    ### regular expression type
    ft['r0'] = sent_re_ft[i]
    if i > 0:
        ft['r-1'] = sent_re_ft[i-1]
    else:
        ft['r-1'] = 'BOS'
    if i < len(sent)-1:
        ft['r+1'] = sent_re_ft[i+1]
    else:        
        ft['r+1'] = 'EOS'
    if i > 1:
        ft['r-2'] = sent_re_ft[i-2]
    else:
        ft['r-2'] = 'BOS'
    if i < len(sent)-2:
        ft['r+2'] = sent_re_ft[i+2]
    else:
        ft['r+2'] = 'EOS'
    ### extra joint feature
    ft['w0+w-1'] = ft['w0'] + ' ' + ft['w-1']
    ft['w0+w+1'] = ft['w0'] + ' ' + ft['w+1']
    ft['w0+s-1'] = ft['w0'] + ' ' + ft['s-1']
    ft['r0+r-1'] = ft['r0'] + ' ' + ft['r-1']
    ft['r0+r+1'] = ft['r0'] + ' ' + ft['r+1']
    ft['w0+r0']  = ft['w0'] + ' ' + ft['r0']
    ft['w0+r-1'] = ft['w0'] + ' ' + ft['r-1']
    ft['w0+r+1'] = ft['w0'] + ' ' + ft['r+1']
    return ft
re_adm_div      = ['ấp', 'buôn', 'bản', 'huyện', 'làng', 'miền', 'nước', 
                   'phường', 'quận', 'tỉnh', 'thành_phố', 'thị_trấn', 'thị_xã', 
                   'thôn', 'TT', 'TP', 'TX', 'TT.', 'TP.', 'TX.', 'xứ', 'xã', 
                   'xóm']
re_org          = ['báo', 'bệnh_viện', 'bệnh_xá', 'công_ty', 'công_ti', 'đài', 'đảng', 'đoàn', 'hội', 'hợp_tác_xã', 'khách_sạn', 'nhà_máy', 'nhà_xuất_bản', 'ngân_hàng', 'quỹ', 'tạp_chí', 'tập đoàn', 'thông_tấn_xã', 'tờ', 'trạm_xá', 'xí_nghiệp','ủy_ban']
re_school       = ['mẫu_giáo', 'tiểu_học', 'trung_học', 'trung_học_cơ_sở', 
                   'trung_học_phổ_thông', 'cao_đẳng', 'trung_cấp', 
                   'trung_cấp_nghề', 'đại_học']
re_street       = ['đại_lộ', 'đường', 'hẻm', 'ngách', 'ngõ', 'nhà', 'phố', 'quốc_lộ']
re_place        = ['ao', 'am', 'bến', 'bến_cảng', 'bến_phà','biển', 'cảng', 
                   'cầu', 'công_viên', 'chợ', 'chùa', 'dãy', 'đảo', 'đầm', 'đèo', 
                   'đền', 'đình', 'đồi', 'động', 'đồng_bằng', 'gềnh', 'gò', 'khu', 'hòn', 'hồ', 
                   'lăng', 'miếu', 'miền', 'nhà_ga', 'núi', 'phà', 'quần_đảo', 
                   'sân_bay', 'sông', 'suối', 'vùng']
re_office       = ['ban', 'bộ', 'chi_cục', 'cục', 'hạt', 'sở']
re_army         = ['binh_đoàn', 'đại_đội', 'đặc_khu', 'đơn_vị', 'lữ_đoàn', 'quân_đoàn', 'quân_đội', 'quân_khu','sư_đoàn', 'tiểu_đội', 'tiểu_đoàn', 'trung_đội']

def re_word(word):
    """
        Return a dict of (regexp Name, regexp Value) of a word
        :type word: string
        :param word: a word in sentence
    """

    check_code = False
    for char in word:
        if char.isdigit():
            check_code = True
            break

    re_dict = dict()
    re_dict['org'] = word.lower() in re_org
    re_dict['name'] = word[0].isupper()
    re_dict['capital'] = word.isupper()
    re_dict['adm_div'] = word.lower() in re_adm_div
    re_dict['is_school'] = word.lower() == 'trường'
    re_dict['school'] = word.lower() in re_school
    re_dict['street'] = word.lower() in re_street
    re_dict['digit'] = word.isdigit()
    re_dict['code'] = check_code
    re_dict['place'] =  word in re_place
    re_dict['office'] = word in re_office
    re_dict['army'] = word in re_army
    return re_dict 

re_type_name = [ 
    ('ofice_name_admdiv_name', ['office', 'name', 'adm_div', 'name']),
    ('school_type_name_name'     , ['is_school', 'school', 'name', 'name']),
    ('school_capital_name_name'  , ['is_school', 'capital', 'name', 'name']),
    ('org_cap_name_name'         , ['org', 'capital', 'name', 'name']),
    ('org_adm_div'          , ['capital', 'adm_div', 'name']),
    ('school_type_name'     , ['is_school', 'school', 'name']),
    ('school_capital_name'  , ['is_school', 'capital', 'name']),
    ('org_cap_name'         , ['org', 'capital', 'name']),
    ('place_name_name'           , ['place', 'name', 'name']),
    ('place_name'           , ['place', 'name']),
    ('org_name'                  , ['org', 'name', 'name']),
    ('school_name_name'          , ['school', 'name', 'name']),
    ('office_name_name'               , ['office', 'name', 'name']),
    ('street_name_name'          , ['street', 'name', 'name']),
    ('org'                  , ['org', 'name']),
    ('school_name'          , ['school', 'name']),
    ('adm_div'              , ['adm_div', 'name']),
    ('office_name'               , ['office', 'name']),
    ('street_name'          , ['street', 'name']),
    ('street_digit'         , ['street', 'digit']),
    ('street_code'          , ['street', 'code']),
    ('army_name'            , ['army', 'code']),
    ('army_name'            , ['army', 'digit']),
    ('army_name'            , ['army', 'name'])
]

def sent_re_feature(sent):
    l = len(sent)
    sent_re_ft = ['NA'] * l
    re_dict_word = [re_word(word[0]) for word in sent]
    for type_name in re_type_name:
        tl = len(type_name[1])
        for i in range(l):
            if (i + tl <= l):
                if (set(['NA']) == set([sent_re_ft[i+j] for j in range(tl)])) and (set([True]) == set([re_dict_word[i+ll][type_name[1][ll]] for ll in range(tl)])):
                    for k in range(tl):
                        sent_re_ft[i+k] = type_name[0]
    return sent_re_ft

def sent_feature_train(sent):
    sent_ft_train = list()
    sent_re_ft = sent_re_feature(sent)
    for i in range(len(sent)):
        if i < 1:
            sent_ft_train.append((word_feature(sent, i, 'BOS', 'BOS', sent_re_ft),
                                  labels_dict[sent[i][3]],
                                  len(labels)))
        elif i < 2:
            sent_ft_train.append((word_feature(sent, i, sent[i-1][3], 'BOS', sent_re_ft),
                                  labels_dict[sent[i][3]],
                                  labels_dict[sent[i-1][3]]))
        else:
            sent_ft_train.append((word_feature(sent, i, sent[i-1][3], sent[i-2][3], sent_re_ft),
                                  labels_dict[sent[i][3]],
                                  labels_dict[sent[i-1][3]]))    
    return sent_ft_train        
    

In [0]:
def sent_feature_test(sent, pre_state, pre_pre_state):
    sent_ft_test = list()
    sent_re_ft = sent_re_feature(sent)
    for i in range(len(sent)):
        sent_ft_test.append(word_feature(sent, i, pre_state, pre_pre_state, sent_re_ft))    
    return sent_ft_test      

In [0]:
def viterbi_decoder(model, sent):
    sent_re_ft = sent_re_feature(sent)
    alpha = [([None] * len(labels)) for i in range(len(sent))]
    trace = np.full(shape=(len(sent), len(labels)), fill_value=-1)

    # start probability
    pdist = model[len(labels)].prob_classify(word_feature(sent, 0, 'BOS', 'BOS', sent_re_ft))    
    alpha[0] = [pdist.prob(l) for l in labels]
    
    for i in range(1, len(sent)):
        alpha[i] = [0] * len(labels)
        for j in range(len(labels)):
            pre_state = labels[j];
            pre_pre_state = 'BOS'
            if i > 1:
                pre_pre_state = labels[trace[i-1][j]];
            feature = word_feature(sent, i, pre_state, pre_pre_state, sent_re_ft)
            pdist = model[j].prob_classify(feature)                
            posterior = [pdist.prob(l) for l in labels]
            for k in range(len(labels)):
                if alpha[i][k] < (posterior[k] * alpha[i-1][j]):
                    alpha[i][k] = posterior[k] * alpha[i-1][j]
                    trace[i][k] = j
    m = alpha[-1][0]
    idx = 0
    for i in range(1, len(alpha[-1])):
        if (alpha[-1][i] > m):
            m = alpha[-1][i]
            idx = i;
    predict = list()
    for i in range(len(sent)-1, -1, -1):
        predict.append(labels[idx])
        idx = trace[i][idx]
    # print(alpha)
    return reversed(predict)

In [0]:
def predict_sent(model, sent):
    y_test_sent = [sent[i][3] for i in range(len(sent))]   
    y_pred_sent = viterbi_decoder(model, sent)
    return y_test_sent, y_pred_sent

In [0]:
def predict(model, sents):
    y_test = []
    y_pred = []
    for sent in sents:
        test, pred = predict_sent(model, sent)
        y_test.extend(test)
        y_pred.extend(pred)
    return y_test, y_pred            

In [0]:
all_data, pos_tag, chunk_tag = prepare_data(data_path)
train_sents, test_sents = train_test_split(all_data, test_size = 0.15, random_state=42)
print("train_sents", len(train_sents))
print("test_sents", len(test_sents))

train_sents 14087
test_sents 2486


In [0]:
train_data = [[] for i in range(len(labels) + 1)]
for sent in train_sents:
    for feature, label, pre_label in sent_feature_train(sent):
        train_data[pre_label].append((feature, labels[label]))     

for i in range(len(labels) + 1):
    print('train_data[' + str(i) +'] length', len(train_data[i]))
    print(train_data[i][0])

train_data[0] length 6259
({'w0': 'về', 's-1': 'B-PER', 's-2': 'O', 'bias': 1, 'is_lower': True, 'is_capital': False, 'is_title': False, 'is_mix': True, 'is_capital_period': False, 'is_digit': False, 'end_digit': False, 'has_hyphen': False, 'is_code': False, 'num_syllabus': 1, 'is_name': False, 'w-1': 'Phong', 'w-2': 'ông', 'w+1': 'Thái_Bình', 'w+2': 'thăm', 'r0': 'NA', 'r-1': 'NA', 'r+1': 'NA', 'r-2': 'NA', 'r+2': 'NA', 'w0+w-1': 'về Phong', 'w0+w+1': 'về Thái_Bình', 'w0+s-1': 'về B-PER', 'r0+r-1': 'NA NA', 'r0+r+1': 'NA NA', 'w0+r0': 'về NA', 'w0+r-1': 'về NA', 'w0+r+1': 'về NA'}, 'O')
train_data[1] length 2899
({'w0': 'Bình', 's-1': 'I-PER', 's-2': 'B-PER', 'bias': 1, 'is_lower': False, 'is_capital': False, 'is_title': True, 'is_mix': True, 'is_capital_period': False, 'is_digit': False, 'end_digit': False, 'has_hyphen': False, 'is_code': False, 'num_syllabus': 1, 'is_name': True, 'w-1': 'Trọng', 'w-2': 'Hồ', 'w+1': 'nói', 'w+2': ':', 'r0': 'NA', 'r-1': 'NA', 'r+1': 'NA', 'r-2': 'NA'

In [0]:
%%time 
max_iter = 10
model = [[] for i in range(len(labels) + 1)]
for i in range(len(labels) + 1):
    if i == 6:
        continue
    print("Training with pre_state", i)
    encoding = BinaryMaxentFeatureEncoding.train(train_data[i], count_cutoff=3, labels = labels, alwayson_features=True)
    model[i]= MaxentClassifier.train(train_data[i], algorithm = 'iis', trace=3, encoding=encoding, max_iter=max_iter)
print("Training with pre_state", 6)
encoding_ = BinaryMaxentFeatureEncoding.train(train_data[6], count_cutoff=4, labels = labels, alwayson_features=True)
model[6] = MaxentClassifier.train(train_data[6], algorithm = 'iis', trace=3, encoding=encoding, max_iter=max_iter)
# save model
pickle.dump(model, open(model_path + "mem-multi-classifier-featureset3-binaryfeature-maxiter10.model", "wb"))

Training with pre_state 0
  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.714
             2          -0.31826        0.923
             3          -0.20834        0.987
             4          -0.14433        0.988
             5          -0.10556        0.990
             6          -0.08088        0.990
             7          -0.06443        0.991
             8          -0.05301        0.991
             9          -0.04482        0.992
         Final          -0.03876        0.993
Training with pre_state 1
  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.614
             2          -0.39534        0.962
             3          -0.26964        0.964
             4          -0.19779        0.968
             5          -0.15380        0.972
          

In [0]:
%%time
# test model
test_model = pickle.load(open(model_path + "mem-multi-classifier-featureset3-binaryfeature-maxiter10.model", "rb"))
y_test, y_pred = predict(test_model, test_sents)
precision, recall, fscore, support = score(y_test, y_pred, labels=eval_labels)
print('labels:    {}'.format(eval_labels))
print('precision: {}'.format([str(round(p*100,2)) + '%' for p in precision]))
print('recall:    {}'.format([str(round(r*100,2)) + '%' for r in recall]))
print('fscore:    {}'.format([str(round(f*100,2)) + '%' for f in fscore]))
print('support:   {}'.format(support))
total_precision = metrics.precision_score(y_test, y_pred, average='weighted', labels=eval_labels)
total_recall = metrics.recall_score(y_test, y_pred, average='weighted', labels=eval_labels)
total_fscore = metrics.f1_score(y_test, y_pred, average='weighted', labels=eval_labels)
print('total precision (weighted): {}'.format(str(round(total_precision*100,2)) + '%'))
print('total recall (weighted): {}'.format(str(round(total_recall*100,2)) + '%'))
print('total fscore (weighted): {}'.format(str(round(total_fscore*100,2)) + '%'))

labels:    ['B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
precision: ['81.97%', '88.03%', '75.31%', '81.44%', '74.42%', '77.78%']
recall:    ['65.37%', '82.51%', '20.71%', '31.34%', '19.39%', '16.84%']
fscore:    ['72.74%', '85.18%', '32.49%', '45.26%', '30.77%', '27.68%']
support:   [1057  526  869  434  165  291]
total precision (weighted): 80.39%
total recall (weighted): 45.54%
total fscore (weighted): 54.67%
CPU times: user 1min 6s, sys: 46.8 ms, total: 1min 6s
Wall time: 1min 6s


In [0]:
print('model 0______________________________________________________')
test_model[0].show_most_informative_features()
print('model 1______________________________________________________')
test_model[1].show_most_informative_features()
print('model 2______________________________________________________')
test_model[2].show_most_informative_features()
print('model 3______________________________________________________')
test_model[3].show_most_informative_features()
print('model 4______________________________________________________')
test_model[4].show_most_informative_features()
print('model 5______________________________________________________')
test_model[5].show_most_informative_features()
print('model 6______________________________________________________')
test_model[6].show_most_informative_features()
print('model 7______________________________________________________')
test_model[7].show_most_informative_features()

model 0 ______________________________________________________
    -inf label is 'B-ORG'
    -inf label is 'I-ORG'
    -inf label is 'B-LOC'
    -inf label is 'I-LOC'
  -2.209 is_title==True and label is 'O'
  -2.013 is_name==True and label is 'O'
  -1.733 is_lower==True and label is 'I-PER'
  -1.041 has_hyphen==True and label is 'I-PER'
  -1.015 num_syllabus==2 and label is 'I-PER'
  -1.010 w-1=='Chương' and label is 'I-PER'
model 1______________________________________________________
    -inf label is 'B-PER'
    -inf label is 'B-ORG'
    -inf label is 'I-ORG'
    -inf label is 'I-LOC'
  -2.500 is_title==True and label is 'O'
   2.415 w-2=='Lý' and label is 'B-LOC'
  -1.949 is_name==True and label is 'O'
   1.813 w+2==':' and label is 'B-LOC'
  -1.119 w-2=='Văn' and label is 'I-PER'
  -1.109 w+2=='tuổi' and label is 'I-PER'
model 2______________________________________________________
    -inf label is 'I-PER'
   1.895 w-2=='thuộc' and label is 'I-LOC'
  -1.891 is_name==True and lab

In [0]:
%%time 
max_iter = 50
model = [[] for i in range(len(labels) + 1)]
for i in range(len(labels) + 1):
    if i == 6:
        continue
    print("Training with pre_state", i)
    encoding = BinaryMaxentFeatureEncoding.train(train_data[i], count_cutoff=3, labels = labels, alwayson_features=True)
    model[i]= MaxentClassifier.train(train_data[i], algorithm = 'iis', trace=3, encoding=encoding, max_iter=max_iter)
print("Training with pre_state", 6)
encoding_ = BinaryMaxentFeatureEncoding.train(train_data[6], count_cutoff=4, labels = labels, alwayson_features=True)
model[6] = MaxentClassifier.train(train_data[6], algorithm = 'iis', trace=3, encoding=encoding, max_iter=max_iter)
# save model
pickle.dump(model, open(model_path + "mem-multi-classifier-featureset3-binaryfeature-maxiter50.model", "wb"))

  ==> Training (50 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.714
             2          -0.31826        0.923
             3          -0.20834        0.987
             4          -0.14433        0.988
             5          -0.10556        0.990
             6          -0.08088        0.990
             7          -0.06443        0.991
             8          -0.05301        0.991
             9          -0.04482        0.992
            10          -0.03876        0.993
            11          -0.03415        0.993
            12          -0.03058        0.993
            13          -0.02774        0.993
            14          -0.02546        0.994
            15          -0.02358        0.994
            16          -0.02203        0.994
            17          -0.02071        0.994
            18          -0.01960        0.994
            19          -0.01864        0.995
  

In [0]:
%%time
# test model
test_model = pickle.load(open(model_path + "mem-multi-classifier-featureset3-binaryfeature-maxiter50.model", "rb"))
y_test, y_pred = predict(test_model, test_sents)
precision, recall, fscore, support = score(y_test, y_pred, labels=eval_labels)
print('labels:    {}'.format(eval_labels))
print('precision: {}'.format([str(round(p*100,2)) + '%' for p in precision]))
print('recall:    {}'.format([str(round(r*100,2)) + '%' for r in recall]))
print('fscore:    {}'.format([str(round(f*100,2)) + '%' for f in fscore]))
print('support:   {}'.format(support))
total_precision = metrics.precision_score(y_test, y_pred, average='weighted', labels=eval_labels)
total_recall = metrics.recall_score(y_test, y_pred, average='weighted', labels=eval_labels)
total_fscore = metrics.f1_score(y_test, y_pred, average='weighted', labels=eval_labels)
print('total precision (weighted): {}'.format(str(round(total_precision*100,2)) + '%'))
print('total recall (weighted): {}'.format(str(round(total_recall*100,2)) + '%'))
print('total fscore (weighted): {}'.format(str(round(total_fscore*100,2)) + '%'))

labels:    ['B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG']
precision: ['74.94%', '85.84%', '69.44%', '71.67%', '68.57%', '73.95%']
recall:    ['91.96%', '89.92%', '60.41%', '39.63%', '29.09%', '30.24%']
fscore:    ['82.58%', '87.84%', '64.62%', '51.04%', '40.85%', '42.93%']
support:   [1057  526  869  434  165  291]
total precision (weighted): 74.4%
total recall (weighted): 68.16%
total fscore (weighted): 69.13%
CPU times: user 1min 5s, sys: 46.9 ms, total: 1min 5s
Wall time: 1min 5s


In [0]:
print('model 0______________________________________________________')
test_model[0].show_most_informative_features()
print('model 1______________________________________________________')
test_model[1].show_most_informative_features()
print('model 2______________________________________________________')
test_model[2].show_most_informative_features()
print('model 3______________________________________________________')
test_model[3].show_most_informative_features()
print('model 4______________________________________________________')
test_model[4].show_most_informative_features()
print('model 5______________________________________________________')
test_model[5].show_most_informative_features()
print('model 6______________________________________________________')
test_model[6].show_most_informative_features()
print('model 7______________________________________________________')
test_model[7].show_most_informative_features()

model 0______________________________________________________
    -inf label is 'B-ORG'
    -inf label is 'I-ORG'
    -inf label is 'B-LOC'
    -inf label is 'I-LOC'
  -4.473 is_title==True and label is 'O'
  -3.940 is_name==True and label is 'O'
  -3.250 is_lower==True and label is 'I-PER'
   2.859 w+1=='quê' and label is 'I-PER'
  -2.139 w-1=='Tuấn' and label is 'I-PER'
  -1.833 w-1=='Chương' and label is 'I-PER'
model 1______________________________________________________
    -inf label is 'B-PER'
    -inf label is 'B-ORG'
    -inf label is 'I-ORG'
    -inf label is 'I-LOC'
   6.363 w-2=='Lý' and label is 'B-LOC'
  -6.073 is_title==True and label is 'O'
   5.003 w+2==':' and label is 'B-LOC'
  -4.113 is_name==True and label is 'O'
  -3.277 w+1=='”' and label is 'O'
   3.068 w0+w+1=='Hồng ,' and label is 'O'
model 2______________________________________________________
    -inf label is 'I-PER'
  -4.443 is_name==True and label is 'O'
   4.235 num_syllabus==3 and label is 'B-LOC'
   